In [1]:
import os
import sys
import math
import operator
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D, Merge
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten, RepeatVector
from keras.layers import MaxPool2D
from keras.models import Sequential 
from keras.layers import Reshape
from collections import OrderedDict
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Permute
from keras.models import load_model, save_model
os.chdir("../../")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
           
        return [np.array([np.transpose(tif.imread(file_name)/255.0,(1,2,0))
                         [self.cp:-self.cp,self.cp:-self.cp,:] for file_name in batch_x]),
               np.array([np.transpose(tif.imread(file_name)/255.0,(1,2,0))
                         [self.cp:-self.cp,self.cp:-self.cp,:] for file_name in batch_x]),
                np.array([np.transpose(tif.imread(file_name)/255.0,(1,2,0))
                         [self.cp:-self.cp,self.cp:-self.cp,:] for file_name in batch_x])], np.array(batch_y)         

class CNN_Model:
    
    def __init__(self, directory):
        
        self.onehot = {}
        self.path = directory
        
        df = pd.read_csv("occurrences_train.csv",low_memory=False)
        with open("Data/hierarchy_data.pkl","rb") as f:
            hd = pkl.load(f)
        with open("Data/class_encoding.pkl","rb") as f:
            self.classes = pkl.load(f)
        with open("Data/order_encoding.pkl","rb") as f:
            self.orders = pkl.load(f)
        with open("Data/family_encoding.pkl","rb") as f:
            self.families = pkl.load(f)
        with open("Data/genus_encoding.pkl","rb") as f:
            self.genuses = pkl.load(f)
        with open("Data/specie_encoding.pkl","rb") as f:
            self.species = pkl.load(f)

        self.onehot_output()

        self.train_pathdata_x = []
        self.train_seq_y = []
        self.test_pathdata_x = []
        self.test_seq_y = []
        
        for cls in hd.keys():
            for order in hd[cls].keys():
                for family in hd[cls][order].keys():
                    for genus in hd[cls][order][family].keys():
                        for specie in hd[cls][order][family][genus]:
                            for im in os.listdir(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.train_pathdata_x.append(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                             +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
                                                             +im)
                                
                            for im in os.listdir(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.test_pathdata_x.append(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                             +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
                                                             +im)
        
        np.random.shuffle(self.train_pathdata_x)
        np.random.shuffle(self.test_pathdata_x)
        
        for p in self.train_pathdata_x:
            y = p.split("/")
            c = int(y[3])
            o = int(y[4])
            f = int(y[5])
            g = int(y[6])
            s = int(y[7])
            self.train_seq_y.append([[c],[o],[f],[g],[s]])
            
        for p in self.test_pathdata_x:
            y = p.split("/")
            c = int(y[3])
            o = int(y[4])
            f = int(y[5])
            g = int(y[6])
            s = int(y[7])
            self.test_seq_y.append([[c],[o],[f],[g],[s]])
    
    def onehot_output(self):
        for sp in self.species:
            y = np.zeros(len(self.species))
            y[list(self.species).index(sp)] = 1
            self.onehot[sp] = y
            
    def model_create(self, time_steps=5, batch_size=32):
        
        cls1 = Sequential()
        cls2 = Sequential()
        cls3 = Sequential()
        # Branch 1
        cls1.add(Conv2D(filters=66, kernel_size=(2, 2), input_shape=(32,32,33), activation = 'relu'))
        cls1.add(Conv2D(filters=96, kernel_size=(3, 3), activation = 'relu'))
        cls1.add(Flatten())
        # Branch 2
        cls2.add(Conv2D(filters=66, kernel_size=(2,2), input_shape=(32,32,33), activation = 'relu'))
        cls2.add(Conv2D(filters=96, kernel_size=(4,4), activation = 'relu'))
        #cls2.add(Conv2D(filters=128, kernel_size=(4,4), activation = 'relu'))
        cls2.add(Flatten())
        # Branch 3
        cls3.add(Conv2D(filters=66, kernel_size=(2,3), input_shape=(32,32,33), activation = 'relu'))
        cls3.add(Conv2D(filters=96, kernel_size=(3,3), activation = 'relu'))
        cls3.add(Conv2D(filters=128, kernel_size=(5,5), activation = 'relu'))
        cls3.add(Flatten())
        
        classifier = Sequential()
        classifier.add(Merge([cls1,cls2,cls3], mode='concat'))
        classifier.add(RepeatVector(time_steps))
        classifier.add(LSTM(25, return_sequences=True))
        # Step 4 - Full connection
        classifier.add(Dense(128, activation = 'relu'))
        classifier.add(Dense(1, activation = 'relu'))
        # Compiling the CNN
        classifier.compile(optimizer = 'rmsprop', loss = 'mae', metrics = ['mae','mse'])
        classifier.summary()
        return classifier
    
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16, time_steps=5):        
        try:
            classifier = load_model("Code/Models/CNN-RNN_1.h5")
        except:
            print("Training")
            classifier = self.model_create(time_steps=time_steps, batch_size=batch_size)
            train_data = ImageDataGenerator(self.train_pathdata_x, self.train_seq_y, batch_size, crop_size)
            history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=True,shuffle=True)
            classifier.save("Code/Models/CNN-RNN_1.h5")
        print("Testing")
        test_data = ImageDataGenerator(self.test_pathdata_x, self.test_seq_y, batch_size, crop_size)
        scores = classifier.evaluate_generator(test_data, use_multiprocessing=True)
        print("Loss : ", scores[0])
        print("Metrics : ", scores[1:])
        return classifier

In [7]:
ob = CNN_Model("Data/Hierarchial Data/")

In [8]:
model = ob.fit_generator(num_epochs=1, batch_size=100, time_steps=5)

Training


/usr/home/miruna/keras/lib/python3.5/site-packages/ipykernel_launcher.py:116: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_3 (Merge)              (None, 232800)            0         
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 232800)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 25)             23282600  
_________________________________________________________________
dense_5 (Dense)              (None, 5, 128)            3328      
_________________________________________________________________
dense_6 (Dense)              (None, 5, 1)              129       
Total params: 23,839,787
Trainable params: 23,839,787
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
 121/1530 [=>............................] - ETA: 43:04 - loss: 301.3027 - mean_absolute_error: 301.3027 - mean_squared_

ValueError: Input arrays should have the same number of samples as target arrays. Found 80 input samples and 100 target samples.